# Adding Locations

In [1]:
import os
import requests
import json        # just for pretty printing
from dotenv import load_dotenv

load_dotenv()
WEBEX_TOKEN = os.environ.get("WEBEX_TOKEN", 
                             # replace below if you don't have a .env
                             "NWVkODBmZWItMjM0Yy00ZDRlLWFjZmQtZWE4ZGNmN2Q1ZDVhOTFiY2UyYjQtNTNj_P0A1_0ec1bbe0-b5aa-454a-bb5b-aecc4e6bdb4c")
BASE_URI = "https://webexapis.com/v1"

headers = {
    "Authorization": "Bearer " + WEBEX_TOKEN, 
    "Content-Type": "application/json", 
    "Accept": "application/json"
}

### 0. Get Location details
From your organisations building/site management system, this will be specific for your case, but the following data will be needed for each site you are adding:
- full address
- announcement and preferred language
- timezone
- main number (and voicemail number if applicable)


### 1. Add the physical location
The tricky part is to convert your location list to pretty much exactly match the below format  
The timezone is in string name format, for a list of countries with their respective TZs see [here](https://developer.webex.com/docs/api/guides/webex-for-broadworks-and-wholesale-common-guide#webex-meetings-site-timezone) or you can use packages that will get the string name based on city   
"announcementLanguage" and "preferredLanguage" will need to be determined too. You can use a default, but in some countries a local language might be needed

https://developer.webex.com/docs/api/v1/locations/create-a-location

In [2]:
location_data = {
    "name": "BRU-Brussels22",
    "timeZone": "Europe/Brussels",
    "announcementLanguage": "en_us",
    "preferredLanguage": "en_us",
    "address": {
        "address1": "De Kleetlaan 6",
        "address2": "",
        "city": "Machelen",
        "state": "",
        "postalCode": "1831",
        "country": "BE"
    }
}

try:
    response = requests.post(
        BASE_URI + "/locations",
        headers=headers,
        json=location_data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    new_location = response.json()
    print(json.dumps(new_location, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)


201 Created
{
    "id": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OLzk4YTkxYTk1LWJiMjEtNDQ4YS1hNjU3LTQ3YzI4NDk0MTE3OQ",
    "name": "BRU-Brussels22",
    "orgId": "Y2lzY29zcGFyazovL3VzL09SR0FOSVpBVElPTi8wZWMxYmJlMC1iNWFhLTQ1NGEtYmI1Yi1hZWNjNGU2YmRiNGM",
    "address": {
        "address1": "De Kleetlaan 6",
        "address2": "",
        "city": "Machelen",
        "state": "",
        "postalCode": "1831",
        "country": "BE"
    },
    "preferredLanguage": "en_us",
    "created": "2024-02-05T12:31:23.918066",
    "modified": "2024-02-05T12:31:23.918066",
    "timeZone": "Europe/Brussels"
}


### 2. Activate calling for the location
We use the data returned from above request, but there's a couple of quirks to work out   
(might be fixed by now):

- if "address2" is present in the address, but empty, we need to discard it altogether
- "announcementLanguage" is not returned, but still needed for the voice activation request

https://developer.webex.com/docs/api/v1/location-call-settings/enable-a-location-for-webex-calling

In [3]:
data = new_location

if ("address2" in data["address"] 
    and data["address"]["address2"] == ""):
    data["address"].pop("address2")

data["announcementLanguage"] = location_data["announcementLanguage"]

try:
    response = requests.post(
        BASE_URI + "/telephony/config/locations",
        headers=headers,
        json=data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    result = response.json()
    print(json.dumps(result, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

201 Created
{
    "id": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OLzk4YTkxYTk1LWJiMjEtNDQ4YS1hNjU3LTQ3YzI4NDk0MTE3OQ"
}


### 3. Add trunk to the location
A trunk can be added manually, see 01_*** how to do that. Change the name below as needed       
First we'll find the trunk by name so we can get the needed ID 

In [4]:
trunk_name = "Europe Trunk"

try:
    response = requests.get(
        BASE_URI + "/telephony/config/premisePstn/trunks",
        params={"name": trunk_name},
        headers=headers,
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    trunks = response.json()        # we'll use this later
    print(json.dumps(trunks, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

200 OK
{
    "trunks": [
        {
            "id": "Y2lzY29zcGFyazovL3VzL1RSVU5LL2NjNzEyOWJkLWFmM2UtNDRiNC04ZjExLTk3ODdkYzIwNTg0ZA",
            "name": "Europe Trunk",
            "location": {
                "name": "Amsterdam",
                "id": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OLzg2MjU3ZTY3LTcxZDItNDZkMy05ZWZhLTIxY2U2Mzg2NWNhZQ"
            },
            "inUse": true,
            "trunkType": "REGISTERING",
            "isRestrictedToDedicatedInstance": false
        }
    ]
}


Update the location to add the trunk (that will be used to send PSTN calls to)

https://developer.webex.com/docs/api/v1/location-call-settings/update-location-webex-calling-details

In [5]:
trunk_id = trunks["trunks"][0]["id"]
location_id = data["id"]

# add connection to our location data
data["connection"] = {
    "type": "TRUNK", 
    "id": trunk_id
}

try:
    response = requests.put(
        BASE_URI + f"/telephony/config/locations/{location_id}",
        headers=headers,
        json=data
    )
    response.raise_for_status()
    print(f"{response.status_code} {response.reason}")
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

204 No Content


### 4. Add main number to the location
For the calling to start working we do need to add a main number. This is used for extension-only users, but will also play a role in emergency calling (country dependant)  
This is a 2 step process:
- add the number (assuming use of LGW, if using a PSTN plan, the number needs to be ordered and ready)
- assign the number to the location

https://developer.webex.com/docs/api/v1/numbers/add-phone-numbers-to-a-location

In [6]:
location_id = data["id"]
number_data = {
    "phoneNumbers": ["+3227029923"],    # non-existing for demo
    "state": "ACTIVE"
}

try:
    response = requests.post(
        BASE_URI + f"/telephony/config/locations/{location_id}/numbers",
        headers=headers,
        json=number_data
    )
    response.raise_for_status()
    print(f"{response.status_code} {response.reason}")
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

201 Created


In [9]:
location_id = data["id"]
location_data = data
data.pop("connection")
location_data["callingLineId"] = {
    "name": "Main Number",
    "phoneNumber": number_data["phoneNumbers"][0]
}

try:
    response = requests.put(
        BASE_URI + f"/telephony/config/locations/{location_id}",
        headers=headers,
        json=location_data
    )
    response.raise_for_status()
    print(f"{response.status_code} {response.reason}")
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

204 No Content


### 5. Add Internal Dialing
The last of the mandatory settings for a site to be voice enabled.   
We'll add a (unique) internal dial string for the site, the format needs to be set on global level.    
For this demo, the format is 8 (steering digit) + 3 (prefix) + 4 (extension)

In [10]:
location_id = data["id"]
location_data["routingPrefix"] = "8999"

try:
    response = requests.put(
        BASE_URI + f"/telephony/config/locations/{location_id}",
        headers=headers,
        json=location_data
    )
    response.raise_for_status()
    print(f"{response.status_code} {response.reason}")
except requests.exceptions.HTTPError as error:
    print(error)
    print(response.json())

206 Partial Content


### 6. Add Features (optional)
TBD
examples:
- set voicemail number
- disable international calls
- ...